# Predicción de la supervivencia de pasajeros del desastre del Titanic con aprendizaje automático

Este ejercicio tiene como objetivo predecir el resultado del desastre del Titanic utilizando aprendizaje automático. Los datos proporcionados están divididos en datos de entrenamiento y prueba. El archivo train.csv se utilizará para ajustar los algoritmos de aprendizaje automático que luego se aplicarán al archivo test.csv para predecir quién sobrevivirá. Los conjuntos de datos consisten en diferentes características que se enumeran a continuación:

| Variable | Definición | Clave |
| :- | -: | :-: |
| PClass | Clase de Pasajero | 1=1ra, 2=2da, 3=3ra |
| Sex | Género | |
| Age | Edad en años |
| SibSp | # de hermanos/as / cónyuges a bordo del Titanic  |
| Parch | # de padres / hijos a bordo del Titanic |
| Ticket | Número de boleto |
| Fare | Tarifa del pasajero |
| Cabin | Número de cabina |
| Embarked | Puerto de embarque | C = Cherburgo, Q = Queenstown, S = Southampton |
| Survived | Supervivencia | 0=No, 1=Sí |

Notas sobre las variables

pclass:
1ra = Superior,
2da = Media,
3ra = Inferior

age: La edad es fraccionaria si es menor a 1. Si la edad es estimada, tendrá la forma xx.5

sibsp: <br> El conjunto de datos define las relaciones familiares de esta manera: <br>
Hermano/a = hermano, hermana, hermanastro, hermanastra <br>
Cónyuge = esposo, esposa (amantes y prometidos/as fueron ignorados)

parch: <br> El conjunto de datos define las relaciones familiares de esta manera: <br>
Padre/Madre = madre, padre <br> Hijo/a = hija, hijo, hijastra, hijastro <br> Algunos niños viajaron solo con una niñera, por lo tanto parch=0 para ellos.

Este ejercicio se divide en tres partes, siendo el procesamiento de datos la más larga e importante.
1. Visualización de datos
2. Procesamiento de datos
3. Algoritmos de aprendizaje automático

## Importar bibliotecas

En esta primera parte hay que importar todas las librerías que se usarán. Seaborn y matplotlib se usan para la visualización de datos. Pandas y numpy ayudarán con el procesamiento de datos y álgebra lineal. Finalmente, se importan las bases de datos para los diferentes algoritmos de aprendizaje automático desde sklearn. Opcionalmente, se agrega una base de datos de imágenes con el propósito de añadir imágenes al cuaderno de Jupyter.

In [ ]:
# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# data processing
import pandas as pd

# linear algebra
import numpy as np

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

#Show images
from PIL import Image

# 1. Visualización de datos

La primera parte de visualización de datos está para obtener un entendimiento del conjunto de datos. Solo conociendo con qué estamos trabajando, podemos determinar qué datos necesitamos o no.

Con todas las librerías importadas, podemos comenzar creando nuevas variables "test_df" que contendrá nuestro conjunto de prueba y "train_df" que incluye el conjunto de entrenamiento. Para hacer esto, usamos la función "read_csv" de pandas.

In [ ]:
#Loading the datasets
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")

In [ ]:
train_df.info()

La función .info() en Python muestra información sobre el conjunto de datos. Muestra todas las columnas, el recuento de entradas dentro de las columnas y qué tipo tienen. En este caso, tenemos dos float64, cinco int64 y cinco objetos.

In [ ]:
train_df.describe()

In [ ]:
train_df.head(10)

Al echar un vistazo rápido al conjunto de entrenamiento, se pueden notar algunos problemas. Hay entradas __faltantes__ (NaN = not a number) que deben ser tratadas. Además, muchas entradas __no son numéricas__, lo que significa que deben ser convertidas __en valores numéricos__ para los algoritmos de aprendizaje automático.

Veamos más de cerca qué es lo que realmente falta.

In [ ]:
#Check for NaN or empty entries in trainingsdata
total = train_df.isnull().sum().sort_values(ascending=False)
percent_1 = train_df.isnull().sum()/train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

Con solo dos valores faltantes, la característica Embarked puede completarse fácilmente. La característica Age no es tan simple de completar y la característica Cabin con 687 valores faltantes probablemente será eliminada por completo.

In [ ]:
print(train_df.columns.values)

Pregunta: <br>
Mirando nuevamente todas las características proporcionadas, ¿cuáles crees que no tendrán un efecto en los modelos de predicción?

Respuesta: <br>
PassengerId, Ticket, Name <br>

Comencemos por mirar las otras características. (Primero Age y Sex)

## Age y Sex

Para la visualización gráfica de Age y Sex con respecto a la tasa de supervivencia, usaremos un histograma. Un histograma muestra la distribución de una característica específica. En este caso, es la distribución de pasajeros femeninos y masculinos respecto a su edad y cuántos sobrevivieron.

In [ ]:
survived = 'survived'
not_survived = 'not survived'
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
women = train_df[train_df['Sex']=='female']
men = train_df[train_df['Sex']=='male']

# Gráfico para mujeres
ax = sns.histplot(women[women['Survived']==1].Age.dropna(), bins=18, label=survived, ax=axes[0], kde=False)
ax = sns.histplot(women[women['Survived']==0].Age.dropna(), bins=40, label=not_survived, ax=axes[0], kde=False)
ax.legend()
ax.set_title('Female')

# Gráfico para hombres
ax = sns.histplot(men[men['Survived']==1].Age.dropna(), bins=18, label=survived, ax=axes[1], kde=False)
ax = sns.histplot(men[men['Survived']==0].Age.dropna(), bins=40, label=not_survived, ax=axes[1], kde=False)
ax.legend()
_ = ax.set_title('Male')

Al crear el histograma, se aprecia claramente que los hombres entre 20-40 años tuvieron una mayor probabilidad de supervivencia. Para las mujeres, la mayor probabilidad de supervivencia está entre 14 y 40 años. También es notable que los infantes tienen una tasa de supervivencia más alta.

Pregunta: <br>
¿Qué se puede hacer con la edad para simplificarla aún más?

Respuesta: <br>
Ciertas edades tienen mayores probabilidades de supervivencia que otras -> crear grupos para tener una escala similar a otras características.

## Embarked, Pclass y Sex

Para visualizar la característica Embarked, usaremos FacetGrid. Ayuda a mostrar distribuciones o relaciones entre múltiples variables.

In [ ]:
FacetGrid = sns.FacetGrid(train_df, row='Embarked', height=4.5, aspect=1.6)
FacetGrid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep', order=None, hue_order=None)
FacetGrid.add_legend()

La característica Embarked también muestra tener un gran impacto en la probabilidad de supervivencia de los pasajeros. Los hombres que embarcaron en el puerto C tienen una mayor probabilidad de supervivencia que en el puerto S o Q. Las mujeres, por otro lado, tienen una baja tasa de supervivencia en el puerto C y altas probabilidades en los puertos S y Q.
La clase también parece tener un efecto en la tasa de supervivencia.

In [ ]:
sns.barplot(x='Pclass', y='Survived', data=train_df)

Es inmediatamente evidente que la clase 1 tiene la mayoría de los sobrevivientes y la clase 3, la menor cantidad.

In [ ]:
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

Esta gráfica muestra la influencia de Pclass y destaca la alta tasa de mortalidad de los pasajeros en la clase 3.

## SibSp y Parch

SibSp y Parch son características que muestran a pasajeros y cuántos familiares tienen a bordo. En consecuencia, esta característica debería combinarse en una sola. El siguiente código crea una nueva característica "not_alone" que muestra si un pasajero no está solo.

In [ ]:
data = [train_df, test_df]
for dataset in data:
    dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['relatives'] > 0, 'not_alone'] = 0
    dataset.loc[dataset['relatives'] == 0, 'not_alone'] = 1
    dataset['not_alone'] = dataset['not_alone'].astype(int)

train_df['not_alone'].value_counts()

In [ ]:
axes = sns.pointplot(x='relatives', y='Survived', data=train_df)

La gráfica muestra la tasa de supervivencia en relación con la cantidad de familiares que tiene una persona. Los pasajeros con 1-3 familiares tienen mayores probabilidades de supervivencia.

__Concluyendo la visualización de datos:__
- Buscamos valores vacíos o faltantes (NaN). Deben tratarse después.
    - Cabin 687
    - Age 177
    - Embarked 2
- Buscamos valores que tienen una influencia en la probabilidad de supervivencia.
    - Age y Sex ambos tienen impacto en la supervivencia.
    - Embarked y Pclass también muestran efecto en la supervivencia.
- SibSp y Parch son características similares y deberían combinarse, y también parecen afectar la probabilidad de supervivencia.

# 2. Preprocesamiento de datos

Después de establecer una comprensión de los datos, podemos comenzar a darles la forma que necesitamos. Para los algoritmos de aprendizaje automático, queremos que todo esté en valores numéricos y que sea similar en escala.

## PassengerId

El PassengerId no tiene influencia en la probabilidad de supervivencia. En este caso, eliminaremos la columna del conjunto de entrenamiento.


In [ ]:
train_df = train_df.drop(['PassengerId'], axis=1)

In [ ]:
#Check tickets
train_df['Ticket'].describe()

In [ ]:
train_df = train_df.drop(['Ticket'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)

## Manejo de datos faltantes

### Cabin

Mirando de nuevo las características, tenemos 687 valores faltantes en Cabin, 177 valores en Age y solo 2 en Embarked.

Los valores en Cabin están compuestos por una letra y un número, siendo la letra la cubierta. En lugar de eliminar la característica por completo, solo eliminaremos el número y crearemos una nueva característica llamada "deck", resultante de la letra de Cabin. La letra de la cubierta se convertirá en numérica y los valores faltantes serán 0.

In [ ]:
#Antigua característica 'Cabin'
print(train_df['Cabin'].head(8))

In [ ]:
import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
data = [train_df, test_df]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)

# we can now drop the cabin feature
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

(Comparación entre Cabin y Deck)

In [ ]:
#Nueva característica 'Deck'
print(dataset['Deck'].head(8))

### Age

Para Age, se agregarán nuevos valores derivados de la edad media de la desviación estándar.

In [ ]:
# Solución corregida para imputar edades faltantes en Titanic
data = [train_df, test_df]

# Se procesa cada conjunto (train y test) por separado
for dataset in data:
    # Se calcula la media y desviación estándar del MISMO conjunto que se va a modificar
    mean = dataset["Age"].mean()
    std = dataset["Age"].std()
    is_null = dataset["Age"].isnull().sum()

    # Se generan valores aleatorios según la distribución del conjunto actual
    rand_age = np.random.randint(mean - std, mean + std, size=is_null)

    # Se crea una copia para evitar SettingWithCopyWarning
    age_slice = dataset["Age"].copy()

    # Se reemplaza los valores NaN con los aleatorios generados
    age_slice[np.isnan(age_slice)] = rand_age

    # Se asignan de vuelta al dataset y convertimos a entero
    dataset["Age"] = age_slice.astype(int)

# Se verifica que no quedan valores nulos en train_df
print("Valores nulos en Age de train_df:", train_df["Age"].isnull().sum())
print("Valores nulos en Age de test_df:", test_df["Age"].isnull().sum())

# ALTERNATIVA 1: Eliminar filas con valores faltantes
# train_df_clean = train_df.dropna(subset=["Age"])
# test_df_clean = test_df.dropna(subset=["Age"])
# Ventaja: Mantiene la pureza de los datos originales
# Desventaja: Reduce significativamente el tamaño de la muestra

# ALTERNATIVA 2: Imputación por grupos demográficos
# Por ejemplo, usando título (Mr, Mrs, etc) y clase (Pclass)
# train_df["Age"] = train_df.groupby(["Sex", "Pclass"])["Age"].transform(lambda x: x.fillna(x.median()))
# Ventaja: Preserva mejor las relaciones entre variables
# Desventaja: Mayor complejidad de implementación

# ALTERNATIVA 3: Usar algoritmos de imputación avanzados
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=5)
# train_df["Age"] = imputer.fit_transform(train_df[["Age"]])
# Ventaja: Mayor precisión estadística
# Desventaja: Requiere normalización previa de datos

In [ ]:
print(train_df['Age'])

La característica Embarked tiene dos valores faltantes. Estos se completarán con la característica más común. La función .describe() de Python cuenta todos los valores y devuelve la función más importante. Esto aplica solo a valores categóricos. Para valores numéricos, la función .describe() devuelve percentiles y valores medios.

In [ ]:
train_df['Embarked'].describe()
# .describe funciona de manera diferente para series numéricas y categóricas. La siguiente vista sólo se logra con datos categóricos. Si más abajo
# los valores se convierten a numéricos, se calcula el promedio, etc.

Para la característica Embarked, la más común es el puerto S, con 644 entradas.

In [ ]:
common_value = 'S'
data = [train_df, test_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(common_value)

Después de llenar todos los valores faltantes, podemos verificar nuestro conjunto de datos con .info(). Todas las características deberían contener ahora 891 entradas y cero valores faltantes. <br>También podemos detectar las características recién añadidas "not_alone" y "Deck". La imagen importada muestra el conjunto de datos original sin el procesamiento de datos.

In [ ]:
#train_df.info()
#print("")
#img = Image.open('train_df_info.jpg')
#img

Como se mencionó anteriormente, todas las características deberían ser valores numéricos. El siguiente paso muestra la conversión a valores numéricos.

In [ ]:
#Converting "fare" from float64 to int 64
data = [train_df, test_df]

for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)

La parte complicada de los nombres es que hay muchos títulos diferentes, aparte de Mr o Mrs, como "Lady", "Countess" y así sucesivamente. Estos no son numerosos pero aún deben tenerse en cuenta. Los siguientes títulos 'Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer' y 'Dona' se convertirán en un solo título llamado 'Rare'.

'Mlle' y 'Ms' se convertirán en 'Miss' y 'Mme' será 'Mrs'.

Al final, nos quedan cinco nuevos títulos en total -> "Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5

In [ ]:
data = [train_df, test_df]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)

train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

Nuevamente usando la función .map(), definimos male como 0 y female como 1. <br>
Lo mismo se hace para convertir los puertos S, C y Q de la característica Embarked.

In [ ]:
genders = {"male": 0, "female": 1}
data = [train_df, test_df]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [ ]:
ports = {"S": 0, "C": 1, "Q": 2}
data = [train_df, test_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)

Después de que todos los valores se han convertido a numéricos, nos queda el siguiente conjunto de datos de entrenamiento.

In [ ]:
train_df.info()

## Creación de nuevas categorías

Referencia a 1. visualización de datos "Age and Sex" -> Crear nueva categoría <br>
Pregunta: <br>
¿Qué sería importante al crear nuevas clases? <br>

Respuesta: <br>
Dividirlas en grupos uniformes, para que ningún grupo incluya demasiadas características en comparación con los otros.

In [ ]:
test = pd.qcut(train_df['Age'], 8)

In [ ]:
test.value_counts()

In [ ]:
#Creating Categories
data = [train_df, test_df]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 21), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 21) & (dataset['Age'] <= 24), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 24) & (dataset['Age'] <= 28), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 28) & (dataset['Age'] <= 32), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 37), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 37) & (dataset['Age'] <= 45), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 45, 'Age'] = 7

# let's see how it's distributed
train_df['Age'].value_counts()

Pregunta: <br>
¿Qué característica queda que también podría usar nuevas categorías? <br>

Respuesta: <br>
Fare

In [ ]:
train_df['Fare'].describe()

Usando la función qcut(), pandas puede crear grupos de rangos que tienen cantidades similares de entradas. <br>
Después de eso, podemos recorrer los datos y asignar el grupo a cada valor.

In [ ]:
groups = pd.qcut(train_df['Fare'], 6)

In [ ]:
groups.value_counts()

In [ ]:
data = [train_df, test_df]

for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7) & (dataset['Fare'] <= 8), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 8) & (dataset['Fare'] <= 14), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 14) & (dataset['Fare'] <= 26), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 26) & (dataset['Fare'] <= 52), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 52, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)

In [ ]:
train_df['Fare'].value_counts()

El último paso antes de aplicar los algoritmos de aprendizaje automático es crear dos nuevas características. <br>
Pregunta: ¿cuáles se podrían combinar? (¿o es demasiado difícil?)

In [ ]:
data = [train_df, test_df]
for dataset in data:
    dataset['Age_Class']= dataset['Age']* dataset['Pclass']

In [ ]:
for dataset in data:
    dataset['Fare_Per_Person'] = dataset['Fare']/(dataset['relatives']+1)
    dataset['Fare_Per_Person'] = dataset['Fare_Per_Person'].astype(int)# Let's take a last look at the training set, before we start training the models.
train_df.head(10)

# 3. Algoritmos de Aprendizaje de Máquinas

In [ ]:
## Eliminar la característica "Survived" del conjunto de datos de entrenamiento y usar el conjunto de datos resultante como "Predictor", llámalo X_train.
## Tomar la característica "Survived" del conjunto de datos de entrenamiento como "Respuesta" para tus modelos, llámala Y_train.

# X_train = "_____________("Survived", axis=1)"
X_train = train_df.drop("Survived", axis=1).copy()
# Y_train = "_____________"
Y_train = train_df["Survived"].copy()

## ¡Esto también lo puedes hacer! Borra el #
X_test  = test_df.drop("PassengerId", axis=1).copy()

## Árbol de decisión

In [ ]:
## Aplica el algoritmo de árbol de decisión al conjunto de datos de entrenamiento

## Queremos evaluar la puntuación de cada modelo de predicción, también es suficiente redondear la puntuación a dos decimales
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
print("Decision Tree Accuracy: ", acc_decision_tree)

## Descenso de gradiente estocástico

In [ ]:
## Aplica el algoritmo SGD al conjunto de datos de entrenamiento con un número máximo de iteraciones = 5 y tol = None
sgd = SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, Y_train)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
print("SGD Accuracy: ", acc_sgd)

## Evalúa la puntuación como para el Árbol de decisión

## Random Forest

In [ ]:
## Aplica el algoritmo Random Forest al conjunto de datos de entrenamiento con un número máximo de estimadores = 100

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
print("Random Forest Accuracy: ", acc_random_forest)
## Evalúa la puntuación como para el Árbol de decisión

## Regresión Logística

In [ ]:
## Aplica el algoritmo de Regresión Logística al conjunto de datos de entrenamiento

logreg = LogisticRegression(max_iter=2000)
logreg.fit(X_train, Y_train)
acc_logreg = round(logreg.score(X_train, Y_train) * 100, 2)
print("Logistic Regression Accuracy: ", acc_logreg)
## Evalúa la puntuación como para el Árbol de decisión

## K vecinos más cercanos

In [ ]:
## Aplica el algoritmo KNN al conjunto de datos de entrenamiento con 3 vecinos

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
print("KNN Accuracy: ", acc_knn)
## Evalúa la puntuación como para el Árbol de decisión

Bayes ingenuo gaussiano

In [ ]:
## Aplica el algoritmo Bayes ingenuo gaussiano al conjunto de datos de entrenamiento

gnb = GaussianNB()
gnb.fit(X_train, Y_train)
acc_gnb = round(gnb.score(X_train, Y_train) * 100, 2)
print("Naive Bayes Accuracy: ", acc_gnb)
## Evalúa la puntuación como para el Árbol de decisión

## Perceptrón

In [ ]:
## Aplica el algoritmo de Perceptrón al conjunto de datos de entrenamiento con 5 iteraciones

perceptron = Perceptron(max_iter=5, tol=None)
perceptron.fit(X_train, Y_train)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
print("Perceptron Accuracy: ", acc_perceptron)
## Evalúa la puntuación como para el Árbol de decisión

## Máquina de soporte vectorial lineal

In [ ]:
## Aplica el algoritmo de Máquina de soporte vectorial lineal al conjunto de datos de entrenamiento con 5 iteraciones

linear_svc = SVC(max_iter=5, tol=None)
linear_svc.fit(X_train, Y_train)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
print("Linear SVC Accuracy: ", acc_linear_svc)

## Evalúa la puntuación como para el Árbol de decisión

## Resultados

In [ ]:
## Al final de nuestro primer ejemplo completo de Aprendizaje Automático, queremos saber qué modelo se ajusta mejor a nuestro problema.
## Por lo tanto, cambiar, si es necesario, el código de acuerdo con tus variables dadas para la puntuación

results = pd.DataFrame({
   'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression',
             'Random Forest', 'Naive Bayes', 'Perceptron',
             'Stochastic Gradient Decent',
             'Decision Tree'],
   'Score': [acc_linear_svc, acc_knn, acc_log,
             acc_random_forest, acc_gaussian, acc_perceptron,
             acc_sgd, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head(9)